In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
from pennylane import BasisState, FermionicSingleExcitation, FermionicDoubleExcitation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1



In [2]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.02, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.02, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.02, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4

params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)
eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.02 Bit-Flip Noise Model")
print('eigenvalues:', eig)

/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


Ground state energy: -1.85863291 Ha
p=0.02 Bit-Flip Noise Model
eigenvalues: [tensor([-1.95201451, -1.90577475, -1.80369488, -1.80060531, -1.76252745,
        -1.70223948, -1.6306034 , -1.43177273, -1.42413389, -1.4184963 ,
        -1.41431298, -1.34639899, -1.34535895, -1.30035891, -1.27648693,
        -1.23328783, -1.22015963, -1.21849959, -1.12156141, -1.11828173,
        -1.10556752, -1.02676595, -0.91222043, -0.9067137 , -0.67086553,
        -0.4646521 , -0.44114416], requires_grad=True)]


In [ ]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.04, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.04, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.04, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.04 Bit-Flip Noise Model")
print('eigenvalues:', eig)


Ground state energy: -1.77698509 Ha
p=0.04 Bit-Flip Noise Model
eigenvalues: [tensor([-1.95218294, -1.91105937, -1.80364268, -1.80355277, -1.75524228,
        -1.70223572, -1.631909  , -1.43171334, -1.42402662, -1.4189754 ,
        -1.41671198, -1.34482533, -1.3379739 , -1.30030295, -1.26508956,
        -1.23314065, -1.22045816, -1.21849953, -1.1215823 , -1.11813596,
        -1.10556569, -1.02883949, -0.91222322, -0.90671341, -0.66803899,
        -0.4646608 , -0.44188289], requires_grad=True)]


In [4]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.06, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.06, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.06, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.06 Bit-Flip Noise Model")
print('eigenvalues:', eig)


Ground state energy: -1.70461951 Ha
p=0.06 Bit-Flip Noise Model
eigenvalues: [tensor([-1.95241332, -1.91482855, -1.80641311, -1.80330411, -1.74931895,
        -1.70154205, -1.63263028, -1.43156724, -1.42387951, -1.41960001,
        -1.41869273, -1.34277967, -1.33123711, -1.30017272, -1.25291556,
        -1.23245355, -1.22060482, -1.21849925, -1.12159974, -1.11794894,
        -1.10546003, -1.03099834, -0.91199028, -0.90682001, -0.66570228,
        -0.46486837, -0.44274448], requires_grad=True)]


In [5]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.08, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.08, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.08, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.08 Bit-Flip Noise Model")
print('eigenvalues:', eig)


Ground state energy: -1.62236821 Ha
p=0.08 Bit-Flip Noise Model
eigenvalues: [tensor([-1.94509525, -1.838367  , -1.81284274, -1.78082506, -1.77248509,
        -1.64763374, -1.60028413, -1.42427918, -1.42268588, -1.41545269,
        -1.41163185, -1.31682625, -1.23131385, -1.23055099, -1.22581486,
        -1.22109187, -1.17819784, -1.12163142, -1.11939643, -1.01941881,
        -1.00524067, -0.98775384, -0.91151904, -0.68157882, -0.66657081,
        -0.63679934, -0.60763506], requires_grad=True)]


In [6]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=0.1, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.1, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p=0.1, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print("p=0.1 Bit-Flip Noise Model")
print('eigenvalues:', eig)


Ground state energy: -1.57627440 Ha
p=0.1 Bit-Flip Noise Model
eigenvalues: [tensor([-1.93253296, -1.91617319, -1.74146812, -1.71767766, -1.71131794,
        -1.68004277, -1.6354585 , -1.43160634, -1.43006048, -1.42385462,
        -1.42319563, -1.34112397, -1.27738716, -1.27463283, -1.23354795,
        -1.22216464, -1.2111692 , -1.12289816, -1.11836612, -1.02538399,
        -0.99775826, -0.91388389, -0.90998468, -0.77538328, -0.77048064,
        -0.76835208, -0.73790534], requires_grad=True)]
